In [22]:
import napari
import numpy as np
from numpy.lib.format import open_memmap
import myfunctions as mf
import os
import matplotlib.pyplot as plt
from skimage.io import imshow
from skimage.measure import regionprops, label
import seaborn as sns
import pandas as pd
import time as clock
from tqdm import tqdm

In [23]:
# function used to compute the ratio between pixels and physical units in millimeters
# and the ratio between time steps and physical units in seconds
def find_geometry(hypervolume_mask, exp):
    if 'P28A' in exp:
        m_diameter = 18.6 # [mm]
    elif 'P28B' in exp:
        m_diameter = 18.6 # [mm]
    elif 'VCT5A' in exp:
        m_diameter = 18.35 # [mm]
    elif 'VCT5' in exp:
        m_diameter = 18.2 # [mm]
    else:
        raise ValueError('Experiment not recognized')
    # here I find the pixel width of the external shell
    rps = regionprops(hypervolume_mask[0,135])
    shell_index = np.argmax([rp.area for rp in rps])
    pixel_diameter = np.sqrt(rps[shell_index].area_bbox)
    m_z = 12 # total field of view in z direction [mm]
    pixel_z = 280 # total field of view in z direction in pixels
    fps = 20 # frames per second
    # computing the actual quantities
    xy_ratio = m_diameter/pixel_diameter
    z_ratio = m_z/pixel_z
    V_ratio = xy_ratio * xy_ratio * z_ratio
    t_ratio = 1/fps
    radius = pixel_diameter/2
    return xy_ratio, z_ratio, V_ratio, t_ratio, radius



# function used to update the dataframe containing the motion properties of the agglomerates
def update_df(df, hypervolume_mask, time, index, label, counts, slices, radii, center, rescale,
              z_sect_str, r_sect_str, V_ratio, t_ratio, t, prev_labels):
    # evaluating the position of the centroid of the agglomerate
    z, y, x = (np.mean(np.where(hypervolume_mask[time] == label), axis=1) - center) * rescale
    # evaluating the distance of the agglomerate from the central axis of the battery
    r = np.linalg.norm([x, y])
    # assigning the agglomerate to a section of the battery
    for i in range(3):
        if slices[i] <= z and z < slices[i+1]:
            z_sect = z_sect_str[i]
        if radii[i] <= r and r < radii[i+1]:
            r_sect = r_sect_str[i]
    # evaluating the volume of the agglomerate
    V = counts[index] * V_ratio
    # evaluating the velocity and volume expansion rate of the agglomerate if it was present in the previous time instant
    # otherwise set these values to 0
    if label in prev_labels:
        x0, y0, z0 = (df.iloc[prev_labels[label]][['x', 'y', 'z']]).values
        vx, vy, vz = (x-x0)/t_ratio, (y-y0)/t_ratio, (z-z0)/t_ratio
        v = np.linalg.norm([vx, vy, vz])
        dVdt = (V - (df.iloc[prev_labels[label]]['V']))/t_ratio
    else:
        vx, vy, vz, v, dVdt = 0, 0, 0, 0, V/t_ratio
    # adding the row to the dataframe
    df = pd.concat([df, pd.DataFrame([[t, label, x, y, z, r, vx, vy, vz, v, V, dVdt, r_sect, z_sect]],
                                     columns=['t', 'label', 'x', 'y', 'z', 'r', 'vx', 'vy', 'vz', 'v', 'V', 'dVdt', 'r_sect', 'z_sect'])])
    return df



# function returning the dataframe containing the motion properties of the agglomerates
def motion_df(hypervolume_mask, exp):
    print('\nComputing motion matrix...')
    df = pd.DataFrame(columns=['t', 'label', 'x', 'y', 'z', 'r', 'vx', 'vy', 'vz', 'v', 'V', 'dVdt', 'r_sect', 'z_sect'])
    # max_label = np.max(hypervolume_mask)
    n_time_instants = hypervolume_mask.shape[0]
    n_slices = hypervolume_mask.shape[1]
    # the ratios between pixels and physical units are computed
    xy_ratio, z_ratio, V_ratio, t_ratio, radius = find_geometry(hypervolume_mask, exp)
    center = np.array([0, 249.5, 249.5])
    rescale = np.array([z_ratio, xy_ratio, xy_ratio])
    # radii and slices are the values used to divide the volume in <steps> regions
    radii = np.linspace(0, radius*xy_ratio, 4)
    radii[3] = np.inf    # the last value is set to inf in order to avoid going out of bounds
    slices = np.linspace(0, n_slices*z_ratio, 4)
    slices[3] = np.inf   # the last value is set to inf in order to avoid going out of bounds
    r_sect_str = ['Core', 'Intermediate', 'External']
    z_sect_str = ['Top', 'Middle', 'Bottom'] # HERE I HAVE TO DOUBLE CHECK THE ORDER OF THE SECTIONS!!!
    current_labels = dict()
    # computing the actual quantities
    for time in tqdm(range(n_time_instants), desc='Computing motion dataframe'):
        prev_labels = current_labels
        current_labels = dict()
        rps = regionprops(hypervolume_mask[time])
        labels = [rp.label for rp in rps]
        counts = [rp.area for rp in rps]
        # converting the time index into seconds
        t = time * t_ratio
        for index, label in enumerate(labels):
            if label > 1:
                current_labels[label] = len(df)
                df = update_df(df, hypervolume_mask, time, index, label, counts, slices, radii, center, rescale,
                            z_sect_str, r_sect_str, V_ratio, t_ratio, t, prev_labels)  
    return df

In [25]:
exp = mf.exp_list()[0]
OS = 'MacOS'
hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask.npy'), mode='r')
center = np.array([0, 249.5, 249.5])
m_diameter = 18.6
rps = regionprops(hypervolume_mask[0,135])
shell_index = np.argmax([rp.area for rp in rps])
pixel_diameter = np.sqrt(rps[shell_index].area_bbox)
m_z = 12
pixel_z = 280
xy_ratio = m_diameter/pixel_diameter
z_ratio = m_z/pixel_z
rescale = np.array([z_ratio, xy_ratio, xy_ratio])

In [28]:
print(rescale)
print(center)

[0.04285714 0.04034707 0.04034707]
[  0.  249.5 249.5]


In [35]:
tic = clock.perf_counter()
z, y, x = (np.mean(np.where(hypervolume_mask[15] == 405), axis=1) - center) * rescale
print(f'time elapsed: {clock.perf_counter() - tic}')

time elapsed: 0.14121983399991223


In [4]:
df = pd.read_csv('/Users/matteoventurelli/Documents/VS Code/MasterThesisData/P28A_FT_H_Exp2/motion_properties.csv')

In [8]:
d = dict()
d[13] = 0

In [20]:
(df.iloc[0]['V'])

1.225798297580004e-10

In [12]:
def OS_path(exp, OS, isrec=False):
    if OS=='Windows':
        if isrec:
            return 'Z:/rot_datasets/selected_vol/' + exp
        else:
            return 'Z:/rot_datasets/' + exp
    elif OS=='MacOS':
        return '../../MasterThesisData/' + exp
    elif OS=='Linux':
        return '/data/projects/whaitiri/Data/Data_Processing_July2022/rot_datasets/' + exp
    elif OS=='Tyrex':
        return 'U:/whaitiri/Data/Data_Processing_July2022/rot_datasets/' + exp
    else:
        raise ValueError('OS not recognized')

In [9]:
def volume_path(exp, time, isrec, isImage=True, OS='Windows'):
    flag = mf.exp_flag()[mf.exp_list().index(exp)]
    vol = '0050' if flag else '0100'
    folder_name = 'entry' + str(time).zfill(4) + '_no_extpag_db' + vol + '_vol'
    volume_name = 'volume_v2.npy' if isImage else 'segmented.npy'
    return os.path.join(OS_path(exp, OS, isrec), folder_name, volume_name)

In [10]:
exp = mf.exp_list()[0]
start_time = mf.exp_start_time()[mf.exp_list().index(exp)]
end_time = 220
skip180=True
OS = 'Windows'
rec = range(123,147)

In [13]:
time_steps = range(start_time, end_time+1, 2) if skip180 else range(start_time, end_time+1)
shape = (len(time_steps), 270, 500, 500)
hypervolume = open_memmap(os.path.join(OS_path(exp, OS), 'hypervolume.npy'), dtype=np.half, mode='w+', shape=shape)
for t, time in tqdm(enumerate(time_steps), desc='Loading hypervolume memmap', total=len(time_steps)):
    volume = open_memmap(volume_path(exp=exp, time=time, OS=OS, isImage=True, isrec=(time in rec)), mode='r')
    hypervolume[t,:,:,:] = volume[10:,208:708,244:744]

Loading hypervolume memmap:   7%|▋         | 4/55 [01:00<12:44, 15.00s/it]

In [ ]:
viewer = napari.Viewer()
images = [viewer.add_image(hypervolume, name='Volume')]